Instalar librerias

Conectar al dataset y descargar spider

In [1]:
import pandas as pd
import json
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


Probar los modelos

In [2]:
import json
import torch
import random
from transformers import BartForConditionalGeneration, BartTokenizer

# Rutas de los modelos y dataset
MODEL_PATH = "/content/drive/MyDrive/bart_nl2sql_finetuned/final_model2"
SPIDER_DATASET_PATH = "/content/drive/MyDrive/spider/test.json"

In [3]:
# Cargar modelo y tokenizer
model = BartForConditionalGeneration.from_pretrained(MODEL_PATH)
tokenizer = BartTokenizer.from_pretrained(MODEL_PATH)
model.eval()

def generate_sql(question, db_id, schema):
    """Genera SQL a partir de una pregunta en lenguaje natural con información del esquema."""
    formatted_input = f"Database: {db_id} | Schema: {schema} | Question: {question}"
    inputs = tokenizer(formatted_input, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=128,
            num_beams=5,  # Evita exploraciones erróneas
            early_stopping=True,
            repetition_penalty=1.8,  # Ajustado para menor penalización
            length_penalty=2.0,  # Ajustado para fomentar estructuras más completas
            no_repeat_ngram_size=4,
            do_sample=False,  # Evita respuestas erráticas
            temperature=0.3  # Reducida para más determinismo
        )
    generated_sql = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Verificar si la salida es válida
    valid_keywords = ["SELECT", "FROM", "WHERE", "GROUP BY", "ORDER BY", "JOIN", "LIMIT"]
    if not any(keyword in generated_sql.upper() for keyword in valid_keywords):
        generated_sql = "INVALID SQL"

    return generated_sql

# Cargar el dataset de Spider
with open(SPIDER_DATASET_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

# Obtener el esquema de la base de datos real
def get_schema(db_id):
    """Extraer esquema real de la base de datos."""
    schema_dict = {
        "db1": "Tables: Rooms, Buildings | Columns: room_size, building_address",
        "db2": "Tables: Customers, Orders | Columns: customer_name, order_quantity, invoice_status",
        "db3": "Tables: Conference, Participants | Columns: conference_id, conference_name, year"
    }
    return schema_dict.get(db_id, "Schema not available")

# Seleccionar 5 preguntas aleatorias
random_samples = random.sample(data, 5)

/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [4]:


# Evaluar para las preguntas aleatorias
results = []
for sample in random_samples:
    question = sample["question"]
    db_id = sample["db_id"]
    expected_sql = sample["query"]
    schema = get_schema(db_id)  # Obtener esquema de la base de datos
    generated_sql = generate_sql(question, db_id, schema)

    accuracy = 1 if generated_sql.strip().lower() == expected_sql.strip().lower() else 0
    results.append({
        "question": question,
        "expected_sql": expected_sql,
        "generated_sql": generated_sql,
        "accuracy": accuracy
    })

# Mostrar los resultados
for result in results:
    print("Pregunta:", result["question"])
    print("SQL Esperado:", result["expected_sql"])
    print("SQL Generado:", result["generated_sql"])
    print("Accuracy:", result["accuracy"])
    print("-" * 50)

# Calcular precisión global
overall_accuracy = sum(r["accuracy"] for r in results) / len(results)
print(f"Accuracy general: {overall_accuracy * 100:.2f}%")


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1528: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Pregunta: Count the number of clubs.
SQL Esperado: SELECT count(*) FROM club
SQL Generado: SELECT count(DISTINCT club) FROM soccer_3
Accuracy: 0
--------------------------------------------------
Pregunta: On which day did the bakery sell more than 100 dollars in total.
SQL Esperado: SELECT T3.date FROM goods AS T1 JOIN items AS T2 ON T1.id  =  T2.item JOIN receipts AS T3 ON T2.receipt  =  T3.ReceiptNumber GROUP BY T3.date HAVING sum(T1.price)  >  100
SQL Generado: SELECT DISTINCT T1.day FROM bakery_1 AS T1 JOIN sales AS T2 ON T1.bakery_id  =  T2.baker_id WHERE T2.total_dollars  >  100
Accuracy: 0
--------------------------------------------------
Pregunta: What is the title and purchase price of the book that has the highest total order amount?
SQL Esperado: SELECT T2.title ,  T2.PurchasePrice FROM Books_Order AS T1 JOIN BOOk AS T2 ON T1.isbn  =  T2.isbn GROUP BY T1.isbn ORDER BY sum(amount) DESC LIMIT 1
SQL Generado: SELECT T1.title, sum(T1.order_amount) FROM book_orders AS T1 JOIN p